In [1]:
from sklearn import datasets, utils
import tensorflow as tf
from sklearn.preprocessing import label_binarize
import numpy as np

In [2]:
iris_dataset = datasets.load_iris()
iris = np.column_stack([iris_dataset['data'], iris_dataset['target']])
iris = utils.shuffle(iris)
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])
data = iris[:, :4]

In [3]:
# Parameters
learning_rate = 0.001
training_epochs = 25
batch_size = 15
display_step = 1

# Network Parameters
n_hidden_1 = 15 # 1st layer number of neurons
n_hidden_2 = 15 # 2nd layer number of neurons
n_input = 4
n_classes = 3 # Iris total classes

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Loop over all batches
        total_batch = int(150/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = iris[i:i+batch_size,:4], target_multiclass[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: data[100:150, :], Y: target_multiclass[100:150]}))

Epoch: 0001 cost=50.709343338
Epoch: 0002 cost=31.646997452
Epoch: 0003 cost=13.049696922
Epoch: 0004 cost=0.864158948
Epoch: 0005 cost=0.255007289
Epoch: 0006 cost=0.417981184
Epoch: 0007 cost=0.225589602
Epoch: 0008 cost=0.081175698
Epoch: 0009 cost=0.089808492
Epoch: 0010 cost=0.096014407
Epoch: 0011 cost=0.082485163
Epoch: 0012 cost=0.085177030
Epoch: 0013 cost=0.086027169
Epoch: 0014 cost=0.082598079
Epoch: 0015 cost=0.083245359
Epoch: 0016 cost=0.082090484
Epoch: 0017 cost=0.081001281
Epoch: 0018 cost=0.080468594
Epoch: 0019 cost=0.079372516
Epoch: 0020 cost=0.078579423
Epoch: 0021 cost=0.077677932
Epoch: 0022 cost=0.076759086
Epoch: 0023 cost=0.075883178
Epoch: 0024 cost=0.074962445
Epoch: 0025 cost=0.074065381
Optimization Finished!
Accuracy: 0.94
